In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from numpy import array
import sys
from PIL import Image 

from keras import layers
#from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
#from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
import random
import pathlib
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.applications.densenet import DenseNet121
import joblib

from keras import backend as K
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, \
    Lambda, MaxPooling2D, Reshape, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam

import os

train_dir = 'train/'
test_dir = 'test/'

In [3]:
# Get the list of training files 
train = os.listdir(train_dir)
# Get the list of test files
test = os.listdir(test_dir)

print("Total number of training images: ",len(train))
print("Toal number of test images: ",len(test))

Total number of training images:  25362
Toal number of test images:  7960


In [4]:
# load training labels into a pandas dataframe
train_labels = pd.read_csv('train.csv')
train_labels.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [5]:
batch_size = 32
img_height = 100
img_width = 100

In [6]:
print("Preparing images")
    
X_train = np.zeros((len(train_labels), img_height, img_width, 1))
count = 0

for fig in train_labels['Image']:
    #load images into images of size 100x100x3
    img = image.load_img(train_dir + fig, color_mode = 'grayscale', target_size=(img_height, img_width, 1))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)
    X_train[count] = x
    if (count%500 == 0):
        print("Processing image: ", count+1, ", ", fig)
    count += 1

count = 0

print("Finished!")

Preparing images
Processing image:  1 ,  0000e88ab.jpg
Processing image:  501 ,  04c72257b.jpg
Processing image:  1001 ,  09cacb84d.jpg
Processing image:  1501 ,  0ef961892.jpg
Processing image:  2001 ,  141b56a1a.jpg
Processing image:  2501 ,  199a417aa.jpg
Processing image:  3001 ,  1ec170983.jpg
Processing image:  3501 ,  23f084b93.jpg
Processing image:  4001 ,  29163ad0b.jpg
Processing image:  4501 ,  2e0fab120.jpg
Processing image:  5001 ,  3347515d9.jpg
Processing image:  5501 ,  3842d71dc.jpg
Processing image:  6001 ,  3d7f4c7d5.jpg
Processing image:  6501 ,  425f763ca.jpg
Processing image:  7001 ,  4714400cd.jpg
Processing image:  7501 ,  4c082fbdf.jpg
Processing image:  8001 ,  50c683e23.jpg
Processing image:  8501 ,  560d986ad.jpg
Processing image:  9001 ,  5b68c83ed.jpg
Processing image:  9501 ,  60410f111.jpg
Processing image:  10001 ,  654951f81.jpg
Processing image:  10501 ,  6a572256c.jpg
Processing image:  11001 ,  6f96f55b6.jpg
Processing image:  11501 ,  74da2b511.jpg

In [7]:
print("Preparing test images")
    
X_test = np.zeros((len(test), img_height, img_width, 1))
count = 0

for fig in test:
    #load images into images of size 100x100x3
    img = image.load_img(test_dir + fig, color_mode = 'grayscale', target_size=(img_height, img_width, 1))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)
    X_test[count] = x
    if (count%500 == 0):
        print("Processing image: ", count+1, ", ", fig)
    count += 1

count = 0

print("Finished!")

Preparing test images
Processing image:  1 ,  00028a005.jpg
Processing image:  501 ,  0f9219c40.jpg
Processing image:  1001 ,  1ea95c247.jpg
Processing image:  1501 ,  2fe4ec2d5.jpg
Processing image:  2001 ,  404206f77.jpg
Processing image:  2501 ,  4fc69fd82.jpg
Processing image:  3001 ,  5ffc2d300.jpg
Processing image:  3501 ,  7042be267.jpg
Processing image:  4001 ,  801c6a722.jpg
Processing image:  4501 ,  906aa36c8.jpg
Processing image:  5001 ,  a0d337cdc.jpg
Processing image:  5501 ,  b1a4e4c18.jpg
Processing image:  6001 ,  c0b378682.jpg
Processing image:  6501 ,  cfe9d9d04.jpg
Processing image:  7001 ,  e00620fc3.jpg
Processing image:  7501 ,  f1b504951.jpg
Finished!


In [8]:
X = X_train/255
X_test = X_test/255
y = train_labels['Id']

values = array(y)
print(values.shape)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
y = onehot_encoded
print(y.shape)
print(X.shape)
print(X_test.shape)

onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(y)

(25361,)
[4786 3808  662 ... 3671 3103    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(25361, 5005)
(25361, 100, 100, 1)
(7960, 100, 100, 1)


In [9]:
# Data augmentation

datagen = image.ImageDataGenerator(
        rotation_range = 40,
        width_shift_range = 0.3,
        height_shift_range = 0.3, 
        horizontal_flip = True, 
        vertical_flip = True,)
datagen.fit(X)

In [25]:
### NEW TRY

def build_head(branch_model, activation='sigmoid'):
    mid = 32
    xa_inp = Input(shape=branch_model.output_shape[1:])
    xb_inp = Input(shape=branch_model.output_shape[1:])
    x1 = Lambda(lambda x: x[0] * x[1])([xa_inp, xb_inp])
    x2 = Lambda(lambda x: x[0] + x[1])([xa_inp, xb_inp])
    x3 = Lambda(lambda x: K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4 = Lambda(lambda x: K.square(x))(x3)
    x = Concatenate()([x1, x2, x3, x4])
    x = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x = Flatten(name='flatten')(x)

    # Weighted sum implemented as a Dense layer.
    x = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')
    return head_model


def build_siamese(branch_model, head_model, optim, img_shape=(img_height, img_width, 1)):
    img_a = Input(shape=img_shape)
    img_b = Input(shape=img_shape)
    xa = branch_model(img_a)
    xb = branch_model(img_b)
    x = head_model([xa, xb])
    model = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['acc'])
    return model

def build_model_pretrained(lr=0.01, img_shape=(img_height, img_width, 1)):
    inp = Input(shape=img_shape)
    inp2 = Input(shape=(img_height, img_width, 3))
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3), input_tensor=inp2)
    x = Conv2D(3,(3,3),padding='same')(inp) 
    x = base_model(x)
    x = GlobalMaxPooling2D()(x)
    branch_model = Model(inputs=inp, outputs=x)
    optim = Adam(lr=lr)
    head_model = build_head(branch_model, activation='sigmoid')
    model = build_siamese(branch_model, head_model, optim, img_shape=img_shape)
    return model, branch_model, head_model

In [28]:
# NEW

mod, branch_model, head_model = build_model_pretrained()

In [10]:
mod = Sequential()
mod.add(DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 1)))
#mod.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (img_height, img_width, 1)))
#mod.add(BatchNormalization(axis = 3, name = 'bn0'))
#mod.add(Activation('relu'))
#mod.add(MaxPooling2D((2, 2), name='max_pool0'))

#mod.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
#mod.add(Activation('relu'))
#mod.add(MaxPooling2D((2, 2), name='max_pool1'))

#mod.add(Conv2D(128, (3, 3), strides = (1,1), name="conv2"))
#mod.add(Activation('relu'))
#mod.add(MaxPooling2D((2, 2), name='max_pool2'))

#mod.add(Conv2D(256, (1, 1), strides = (1,1), name="conv3"))
#mod.add(Activation('relu'))
#mod.add(MaxPooling2D((2, 2), name='max_pool3'))

#mod.add(Conv2D(512, (1, 1), strides = (1,1), name="conv4"))
#mod.add(Activation('relu'))
#mod.add(MaxPooling2D((2, 2), name='max_pool4'))

mod.add(Flatten())
mod.add(Dense(512, activation="relu", name='rl0'))
mod.add(Dense(256, activation="relu", name='rl1'))
mod.add(Dropout(0.5))
mod.add(Dense(128, activation="relu", name='rl2'))
mod.add(Dropout(0.5))
mod.add(Dense(64, activation="relu", name='rl3'))
mod.add(Dropout(0.5))
mod.add(Dense(32, activation="relu", name='rl4'))
mod.add(Dense(y.shape[1], activation='softmax', name='sm'))

print(mod.output_shape)
mod.summary()

#opt = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
mod.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


(None, 5005)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 3, 3, 1024)        7031232   
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
rl0 (Dense)                  (None, 512)               4719104   
_________________________________________________________________
rl1 (Dense)                  (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
rl2 (Dense)                  (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128) 

In [ ]:
history = mod.fit(tuple(datagen.flow(X, y, batch_size=batch_size)), steps_per_epoch=len(X) / 32, epochs=20 , verbose=1)

In [ ]:
# Predicting test images
predictions = mod.predict(X_test)
y_classes = predictions.argmax(axis=-1)
pred_decoded = label_encoder.inverse_transform(y_classes)
print(pred_decoded)

In [ ]:
# Saving predictions to csv
d = {'Image': test, 'Id': pred_decoded}
df = pd.DataFrame(data=d)
df.to_csv('submission.csv',index=False)
df

In [ ]:
joblib.dump(mod,'siamese_densenet121.sav')